# **Tool calling in Writer**

**_Tool calling_** is a way for Palmyra (the Writer family of LLMs) to call on external tools — hence the name — in order to perform specific tasks that the it can’t do by itself. These tasks include things like making API calls, performing calculations, or accessing external information. This cookbook shows how to use tool calling with Writer’s chat completion API to build AI applications that can provide better results than standard LLM-powered apps.

## Contents

- [Introduction](#introduction)
- [Setup](#setup)
- [Define your functions](#define-your-functions)
- [Define the schema for your functions](#define-the-schema-for-your-functions)
- [Build an app that uses tool calling](#build-an-app-that-uses-tool-calling)


## Introduction

### Why would I want to use tool calling?

With tool calling, Palmyra is given a set of tools that it can call on — hence the name — to enhance its capabilities so that it can provide more useful responses to users. These tools are registered with the LLM, so that it “knows” that they’re available. When the LLM receives a request, it “decides” if it needs to call on a tool for extra information or to take some kind of action based on the conversation it’s having.

Some reasons for using tool calling:

- **It allows for more accurate results than unassisted natural language generation can provide.** Tool calling works especially well for accessing up-to-date information, performing calculations or computation, and working with quantities. It can prevent the error many LLMs make when asked [“How many times does the letter ‘r’ appear in ‘strawberry?’”](https://www.inc.com/kit-eaton/how-many-rs-in-strawberry-this-ai-cant-tell-you.html)
- **It extends what the model can do.** With tool calling, a model can provide results that wouldn’t be possible if it used only its training data by calling external APIs or services, accessing databases, or interacting with your organization’s systems.
- **Automation.** Tool calling can automate workflows or trigger actions, making the model useful in a wide range of applications, from e-commerce to customer service.

### Functions as tools

One of the tool categories available to Palmyra are functions — and by functions, we mean the kind used in programming languages. As the developer of a Palmyra-powered application, you would define these functions and then define a _schema_, which is a data structure that acts as a catalog of tools that Palmyra can call for your application. The schema lists the available tools, the information they might require, and the results they produce.

In the case where the tools are functions, the schema specifies the available functions, what arguments they might take, and what values they return.

### How does tool calling work?
Here’s a quick description of how it works:

1. **User request**: The user asks for something that requires an external tool, such as “What's the current weather in Paris?”. 
2. **Model decides**: Based on the user’s input, the LLM recognizes that it needs a tool — in the case of our example, a way to get the weather at a specific location — to fulfill the request.
3. **Tool call**: The LLM generates a command or request to call the appropriate tool, and if necessary, it generates any needed parameters. In our example, the tool is a function that can access a weather API, and the necessary parameter is “Paris”.
4. **Tool response**: The tool processes the request and sends back the needed information. In our example, this information is the current weather in Paris.
5. **Model response**: The LLM uses the information from the tool to generate a response to the user's request.

## **Setup**

### Dependencies

This notebook uses the following packages:

* `geopy`: For the Nominatim geocoder, to convert addresses and place names into latitudes and longitudes.
* `python-dotenv`: To load environment variables.
* `requests`: To make API calls to IpInfo (user location from IP address) and Open-Meteo (weather).
* `writer-sdk`: To makes calls to the Writer API.

Run the cell below ensure you have these packages.

In [ ]:
%pip install -r requirements.txt -q

### Initialization

The cell below performs the initialization required for this notebook including the creation of an instance of the `Writer` object to interact with the LLM.

To create a Writer client object, you need an API key. [You can sign up for one for free](https://app.writer.com/register). 

Once you have an API key, we recommend that you store it as an environment variable in a `.env` file like so:

```
WRITER_API_KEY="{Your Writer API key goes here}"
```

When you instantiate the client with `client = Writer()`, the newly-created object will automatically look for an environment variable named `WRITER_API_KEY` and will complete the instantiation if an only if `WRITER_API_KEY` has been defined. This notebook uses the [python-dotenv] library to automatically define environment variables based on the contents of an `.env` file in the same directory.

The `Writer()` initializer method also has an `api_key` parameter that you can use like this...

```
client = Writer(api_key="{Your Writer API key goes here}")
```

...but we strongly encourage you not to leave API keys in your source code.

In [ ]:
# Run this cell before running any other cells in this cookbook!

from geopy.geocoders import Nominatim
import requests
from writerai import Writer

# Load environment variables from .env file
%reload_ext dotenv
%dotenv

client = Writer()

## Define your functions**

The next step is to define some functions for the model to call. Let’s define two functions:

<table width="66%">
    <tr>
        <th width="25%" style="background-color: #5551ff; color: #ffffff;">Function</th>
        <th style="background-color: #5551ff; color: #ffffff;">Description</th>
    </tr>
    <tr>
        <td style="border: 1px solid #bfcbff;"><code>get_location_from_ip()</code></td>
        <td style="border: 1px solid #bfcbff;">
            <p>This function calls the [IPInfo](https://ipinfo.io/) service to determine the user’s location
            based on their IP address.</p>
            <p>We expect this function to be called when the user asks for their location or asks a question
            along the lines of “Where am I?”</p>
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid #bfcbff;"><code>get_weather_at_location()</code></td>
        <td style="border: 1px solid #bfcbff;">
            <p>This function call the [Open-Meteo](https://open-meteo.com/) API, which takes a latitude and longitude and
            returns the current weather at those coordinates. It also uses the [Nominatim](https://nominatim.org/) geocoder
            to convert place names and addresses into latitudes and longitudes.</p>
        </td>
    </tr>
</table>

### **`get_location_from_ip()`**

Returns the user’s location based on their IP address using the [IPInfo](https://ipinfo.io/) service.

#### Dependencies

This function requires the following packages:

- [`requests`](https://pypi.org/project/requests/)

#### Arguments

None.


#### Returns

Dictionary with the following keys:

- **`"city"`** (`str`): The city in which the user is located if it can be determined, or "Unknown" if not.
- **`"region"`** (`str`): The region (state or province) in which the user is located if it can be determined, or "Unknown" if not.
- **`"country"`** (`str`): The country in which the user is located if it can be determined, or "Unknown" if not.


### Examples

```
>>> get_location_from_ip() # Tampa, Florida, USA
{'city': 'Tampa', 'region': 'Florida', 'country': 'US'}

>>> get_location_from_ip() # VPN server in Milan, Italy
{'city': 'Milan', 'region': 'Lombardy', 'country': 'IT'}
```

In [ ]:
def get_location_from_ip():
    response = requests.get("https://ipinfo.io")
    if response.status_code == 200:
        data = response.json()
        return {
            "city"    : data.get("city", "Unknown"), 
            "region"  : data.get("region", "Unknown"), 
            "country" : data.get("country", "Unknown"),
        }
    else:
        return {
            "city"    : "Unknown", 
            "region"  : "Unknown", 
            "country" : "Unknown",
        }

### **`get_weather_at_location()`**

Returns the current weather for a given location, specifically:

- Current weather conditions (e.g. "clear sky", "overcast", "moderate rain"...)
- Cloud cover percentage
- Temperature (defaults to Celsius)
- Humidity percentage

The weather information is formatted to be easily understood by an LLM (and humans too!) and includes units where appropriate in order to avoid any ambiguity.

This function gets its weather information from [Open-Meteo](https://open-meteo.com/), an open source weather API that can be access for free for non-commercial use. Open-Meteo requires a latitude and longitude as input, so this function uses the [Nominatim](https://nominatim.org/) geocoder, which uses [OpenStreetMap](https://www.openstreetmap.org/) data to convert human-friendly location names (which could be a reasonably well-known place, building, or landmark, a street address, or city) into those values.

#### Dependencies

This function requires the following packages:

- [`geopy`](https://pypi.org/project/geopy/) for the `Nominatim` geocoder (import it like so: `from geopy.geocoders import Nominatim`)
- [`requests`](https://pypi.org/project/requests/)


#### Arguments

- **`location_name`** (`str`): 
Name of the place for which the weather is requested. This can be as specific as an address, but should at least specify a city. It can also be a reasonably well-known place (for example, the roadside attraction [“Carhenge”](https://carhenge.com/) works). Basically, any location known to OpenStreetMap will work.
- **`use_metric_units`** (`bool`, defaults to `False`):
Set to `True` if you want the temperature in Celsius and wind speed in kilometers per hour; otherwise it defaults to returning the temperature in degrees Fahrenheit and wind speed in miles per hour.


#### Returns

Dictionary with the following keys:

- **`"weather"`** (`str`): One of the values from the `WEATHER_CODE_TABLE` dictionary (defined in the function).
- **`"cloud_cover"`** (`str`): Cloud cover, expressed as a percentage (an integer in the 0 - 100 range followed by a "%" character).
- **`"temperature"`** (`str`): Temperature expressed to 1 decimal point of accuracy followed by “degrees C” or “degrees F”, depending on the value of `use_metric_units`.
- **`"wind_speed"`** (`str`): Wind speed expressed to 1 decimal point of accuracy followed by “km/h” or “mph”, depending on the value of `use_metric_units`.
- **`"humidity"`** (`str`): Humidity, expressed as a percentage (an integer in the 0 - 100 range followed by a "%" character).


#### Examples

```
>>> get_weather_at_location("New York City")
{'weather': 'partly cloudy',
 'cloud_cover': '62%',
 'temperature': '66.9 degrees F',
 'wind_speed': '2.5 mph',
 'humidity': '69%'}

>>> get_weather_at_location("Eiffel Tower", use_metric_units=True)
{'weather': 'partly cloudy',
 'cloud_cover': '54%',
 'temperature': '13.8 degrees C',
 'wind_speed': '7.1 km/h',
 'humidity': '93%'}
```

In [ ]:
def get_weather_at_location(location_name, use_metric_units=False):

    WEATHER_CODE_TABLE = {
        0:  "clear sky",
        1:  "mainly clear", 
        2:  "partly cloudy",
        3:  "overcast",
        45: "fog",
        48: "depositing rime fog",
        51: "light drizzle",
        53: "moderate drizzle",
        55: "dense drizzle",
        56: "light freezing drizzle",
        57: "dense freezing drizzle",
        61: "slight rain",
        63: "moderate rain",
        65: "heavy rain",
        66: "light freezing rain",
        67: "heavy freezing rain",
        71: "slight snow",
        73: "moderate snow",
        75: "heavy snow",
        77: "snow grains",
        80: "light rain showers",
        81: "moderate rain showers",
        82: "violent rain showers",
        85: "slight snow showers",
        86: "heavy snow showers",
        95: "thunderstorm",
        96: "thunderstorm with slight hail",
        99: "thunderstorm with heavy hail",
    }

    def location_name_to_latlong(location_name):
        geolocator = Nominatim(user_agent="Writer.com tool calling demo notebook")
        location = geolocator.geocode(location_name)
        return (location.latitude, location.longitude)

    def celsius_to_fahrenheit(degrees_celsius):
        return (degrees_celsius * 1.8) + 32

    def kmh_to_mph(kmh):
        return kmh * 0.621371

    latitude, longitude = location_name_to_latlong(location_name)
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,relativehumidity_2m,weathercode,cloudcover,wind_speed_10m"
    response = requests.get(url)
    data = response.json()
    return {
        "weather"     : WEATHER_CODE_TABLE.get(data["current"]["weathercode"], "unknown"),
        "cloud_cover" : f"{data['current']['cloudcover']}%",
        "temperature" : (f"{data['current']['temperature_2m']:.1f} degrees C" if use_metric_units 
                         else f"{celsius_to_fahrenheit(data['current']['temperature_2m']):.1f} degrees F"),
        "wind_speed"  : (f"{data['current']['wind_speed_10m']:.1f} km/h" if use_metric_units
                         else f"{kmh_to_mph(data['current']['wind_speed_10m']):.1f} mph"),
        "humidity"    : f"{data['current']['relativehumidity_2m']}%",
    }

## **Define the schema for your functions**

In order for the model to call the functions you defined, it needs to know:

* That your application is providing functions that it can call.
* The names of those functions.
* The descriptions of what the functions do or what they are for.
* The parameters that those function accept, and which ones are required.

When the model receives a prompt, it will analyzes that prompt and determine if it can be matched to a function in the schema.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_location_from_ip",
            "description": "Get the user's location based on their IP address. If the user asks where they are or says they're lost, use this.",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather_at_location",
            "description": "Get the current weather for a given location, which may be a street address, the name of a building, landmark, or destination, or a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "A location, such as an address, a city and province or state with country, or even the name of a reasonably well-known place."
                    }
                },
                "required": [
                    "location"
                ]
            }
        }
    }
]

## **Build an app that uses tool calling**

The cell below contains a basic multi-turn chat completion app. When you run it, you will be able to have an ongoing conversation with Palmyra until you enter a blank line, which stops the app. The app displays the number of prompts you have entered so far.

The app maintains a record of the conversation in the `messages` list — both the user’s messages (the ones where the value of the `"role"` key is `"user"`), and Palmyra’s replies (messages where the value of the `"role"` key is `"assistant"`). You can see the contents of `messages` while the app is running by entering `!messages` as a prompt (it will not count as part of the conversation).



In [ ]:
user_prompt_count = 1
initial_system_message = {
    "role": "system",
    "content": "You are a helpful assistant. Respond concisely and politely to user queries. Use clear, simple language. When asked for technical explanations, provide detailed and accurate information, but avoid jargon. If the user asks for assistance with a task, offer step-by-step guidance."
}
messages = [initial_system_message]

print("""
Sample multi-turn chat completion app
featuring tool calling
=====================================
""")
temperature = float(input("Enter a temperature (0.0 - 2.0) for the chat, or just press 'Enter' for 1.0: ").strip() or 1.0)

while True:
    user_prompt = input(f"[{user_prompt_count}]\nEnter a prompt: ").strip()
    if not user_prompt:
        break

    if user_prompt == "!messages":
        print(f"\nContents of `messages` (this will not be included as part of the conversation):")
        print("-------------------------------------------------------------------------------")
        print(f"{messages}\n\n")
        continue

    user_prompt_count +=1
    user_message = {
        "role": "user",
        "content": user_prompt
    }
    messages.append(user_message)

    # Make initial call to chat() function
    # TODO: Replace this with the production SDK call
    initial_response = client.chat.chat(
        model="palmyra-x-004", 
        messages=messages,
        temperature=temperature,
        tools=tools, 
        tool_choice="auto",
        stream=False
    )
    initial_response_message = initial_response.choices[0].message
    messages.append(initial_response_message)

    # Make secondary call to chat() function
    # if Palmyra decides that it needs to call a tool
    tool_calls = initial_response_message.tool_calls
    if tool_calls:
        for tool_call in tool_calls:
            if tool_call.function.name == "get_weather_at_location":
                location = eval(tool_call.function.arguments)["location"]
                function_response = get_weather_at_location(location)
            if tool_call.function.name == "get_location_from_ip":
                function_response = get_location_from_ip()
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": tool_call.function.name,
                "content": str(function_response),
            })

        final_response = client.chat.chat( 
            messages=messages,
            temperature=temperature,
            model="palmyra-x-004",
            stream=False
        )
        final_response_message = {
            "role": final_response.choices[0].message.role,
            "content": final_response.choices[0].message.content
        }
        messages.append(final_response_message)
        print(f"\n{final_response.choices[0].message.content}\n")
    else:
        print(f"\n{final_response.choices[0].message.content}\n")

To learn more about tool calling, check out the [tool calling guide](https://dev.writer.com/api-guides/tool-calling) on the Writer docs.

### Notes

In the code above, there are _two_ calls to the chat completion API’s `chat()` method. The first one always executes:

```python
initial_response = client.chat.chat(
    model="palmyra-x-004", 
    messages=messages,
    temperature=temperature,
    tools=tools, 
    tool_choice="auto"
)
```

This looks like the standard call to `chat()`, but with a couple of extra parameters:

<table width="66%">
    <tr>
        <th width="25%" style="background-color: #5551ff; color: #ffffff;">Parameter</th>
        <th style="background-color: #5551ff; color: #ffffff;">Description</th>
    </tr>
    <tr>
        <td style="border: 1px solid #bfcbff;"><code>tools</code></td>
        <td style="border: 1px solid #bfcbff;">The tool schema.</td>
    </tr>
    <tr>
        <td style="border: 1px solid #bfcbff;"><code>tool_choice</code></td>
        <td style="border: 1px solid #bfcbff;">
            Specifies the tool that Palmyra should use. Most of the time, you should simply set
            this to `auto` to let Palmyra decide.
        </td>
    </tr>
</table>

If Palmyra decides that it needs to call a tool, the response message’s `tool_calls` property will contain a list. The code above iterates through the list to find out which function Palmyra decided to use and creates a new message with a `tool` role whose content is the function’s result. That message gets added to the list of messages for the conversation, which in turn is sent as part of the second call to `chat()`:

```python
final_response = client.chat.chat( 
    messages=messages,
    temperature=temperature,
    model="palmyra-x-004"
)
```

Note that this call _does not_ contain the `tools` or `tool_choice` parameters — it’s a regular `chat()` call. The app then displays the result of this call to `chat()`.